# Baseline
1. 레이블을 제외한 컬럼의 결측치 최빈값으로 처리
2. 질적 변수 수치화
3. 레이블이 없는 데이터 제거

In [ ]:
import pandas as pd
import numpy as np
import random
import os
from sklearn.preprocessing import LabelEncoder

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(42)

In [ ]:
train = pd.read_parquet('./train.parquet').drop("ID", axis=1)
test = pd.read_parquet('./test.parquet').drop("ID", axis=1)
sample_submission = pd.read_csv('./dts/sample_submission.csv')

In [ ]:
NaN_col = []

for i in range(len(train.columns)):
    if(sum(train[train.columns[i]].notna())<1000000):
        NaN_col.append(train.columns[i])
        
print(NaN_col)

for col in NaN_col:
    mode = train[col].mode()[0]
    train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna(mode)

print('Done')

In [ ]:
qual_col = []

for i in range(len(train.dtypes)):
    if (train.dtypes[i] == 'object'):
        qual_col.append(train.columns[i])
        
print(qual_col)
qual_col.remove('Delay')
print(qual_col)

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
            
    test[i] = le.transform(test[i])
    
print('Done.')

In [ ]:
train = train.dropna()

column_number = {}
for i,column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))
print('Done.')

In [ ]:
train_x = train.drop(columns = ['Delay','Delay_num'])
train_y = train['Delay_num']